### import CSV & query basic infomation
Include #event, #cases, the start activities and the end activities

In [86]:
import pandas as pd
import pm4py as pm

training_file_path = '12689204\BPI_Challenge_2012.XE\BPI_Challenge_2012.XE-training.csv'




In [87]:
event_log = pd.read_csv(training_file_path, sep=',')
event_log = pm.format_dataframe(
    event_log, 
    case_id='case concept:name', 
    activity_key='event concept:name', 
    timestamp_key='event time:timestamp'
)
num_events = len(event_log)
num_cases = len(event_log['case concept:name'].unique())
print("Number of events: {}\nNumber of cases: {}".format(num_events, num_cases))
print('=======================================================')

start_activities = pm.get_start_activities(event_log)
end_activities = pm.get_end_activities(event_log)
print("Start activities: {}\nEnd activities: {}".format(start_activities, end_activities))
print('=======================================================')

print('events: ')
print(event_log['event concept:name'].unique())

C:\Users\20211321\AppData\Local\Temp\ipykernel_23192\3438941301.py:2: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  event_log = pm.format_dataframe(


Number of events: 239786
Number of cases: 12478
Start activities: {'A_SUBMITTED': 10097, 'W_Nabellen offertes': 1484, 'W_Completeren aanvraag': 391, 'W_Valideren aanvraag': 299, 'W_Nabellen incomplete dossiers': 124, 'W_Afhandelen leads': 65, 'O_CANCELLED': 9, 'W_Beoordelen fraude': 6, 'A_CANCELLED': 3}
End activities: {'A_DECLINED': 3429, 'W_Completeren aanvraag': 2428, 'W_Afhandelen leads': 2269, 'W_Nabellen offertes': 2033, 'W_Valideren aanvraag': 1417, 'W_Nabellen incomplete dossiers': 399, 'A_CANCELLED': 324, 'O_CANCELLED': 117, 'W_Beoordelen fraude': 59, 'W_Wijzigen contractgegevens': 3}
events: 
['A_SUBMITTED' 'A_PARTLYSUBMITTED' 'A_PREACCEPTED'
 'W_Completeren aanvraag' 'A_ACCEPTED' 'A_FINALIZED' 'O_SELECTED'
 'O_CREATED' 'O_SENT' 'W_Nabellen offertes' 'O_SENT_BACK'
 'W_Valideren aanvraag' 'O_ACCEPTED' 'A_REGISTERED' 'A_APPROVED'
 'A_ACTIVATED' 'O_CANCELLED' 'W_Wijzigen contractgegevens' 'A_DECLINED'
 'A_CANCELLED' 'W_Afhandelen leads' 'O_DECLINED'
 'W_Nabellen incomplete dossi

In [72]:
test_time_range_filtered_log = pm.filter_time_range(event_log, "2011-01-01 00:00:00", "2013-01-01 00:00:00", mode='events')
print(test_time_range_filtered_log)



               eventID  case concept:name                    case REG_DATE  \
0                    0             173688 2011-01-10 00:38:44.546000+00:00   
1                    1             173688 2011-01-10 00:38:44.546000+00:00   
2                    2             173688 2011-01-10 00:38:44.546000+00:00   
3                    3             173688 2011-01-10 00:38:44.546000+00:00   
4                    4             173688 2011-01-10 00:38:44.546000+00:00   
...                ...                ...                              ...   
239781  56203942035460             214376 2012-02-29 23:51:16.799000+00:00   
239782  56203942035461             214376 2012-02-29 23:51:16.799000+00:00   
239783  56203942035456             214376 2012-02-29 23:51:16.799000+00:00   
239784  56203942035457             214376 2012-02-29 23:51:16.799000+00:00   
239785  56203942035458             214376 2012-02-29 23:51:16.799000+00:00   

        case AMOUNT_REQ  event org:resource      event concept:

In [73]:

#extract all the events of a unique case into a separate dataframe
def getCase(df,case_ID):
    df_case = df[df['case concept:name'] == case_ID]
    return df_case


def getNextEventIndex(df_case, currentEventID):
    df = df_case.sort_values(by='eventID')
    token = False
    for index, r in df.iterrows():
        if token:
            return index
        if currentEventID == df['eventID'][index]:
            token = True
    return "N\A"
    
def writeGroundtruth(df):
    df = df.head(1000)
    #add new columns containing the name of the next event in the case and the time when it happens
    df = df.assign(ground_truth_activity='') 
    df = df.assign(ground_truth_time='')

    for ind in df.index:
        case_num = df['case concept:name'][ind]
        df_case = getCase(df, case_num)
        next_event_index = getNextEventIndex(df_case, df_case['eventID'][ind])
        if next_event_index != "N\A":
            df.at[ind,'ground_truth_activity'] = df.loc[next_event_index,'event concept:name']
            df.at[ind,'ground_truth_time'] = df.loc[next_event_index, 'event time:timestamp']
        else:
            df.at[ind,'ground_truth_activity'] = "N\A"
            df.at[ind,'ground_truth_time'] = "N\A"
            
    return df


In [85]:

df_event = writeGroundtruth(event_log)
event_log.head()


,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,case:concept:name,concept:name,time:timestamp,@@index,@@case_index
0,0,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546000+00:00,173688,A_SUBMITTED,2011-01-10 00:38:44.546000+00:00,0,0
1,1,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880000+00:00,173688,A_PARTLYSUBMITTED,2011-01-10 00:38:44.880000+00:00,1,0
2,2,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906000+00:00,173688,A_PREACCEPTED,2011-01-10 00:39:37.906000+00:00,2,0
3,3,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875000+00:00,173688,W_Completeren aanvraag,2011-01-10 00:39:38.875000+00:00,3,0
4,4,173688,2011-01-10 00:38:44.546000+00:00,20000,NaN,W_Completeren aanvraag,START,2011-01-10 11:36:46.437000+00:00,173688,W_Completeren aanvraag,2011-01-10 11:36:46.437000+00:00,4,0


In [75]:
def computeTimeDifference(df):
    df = df.head(1000)
    df['time_until_next_event'] = 0  # initialize new column with zeros
    
    # iterate over each row of the dataframe
    for i, row in df.iterrows():
        currentCase = row['case concept:name']
        
        # check if there is a next row with the same case
        if i < len(df)-1 and df.loc[i+1, 'case concept:name'] == currentCase:
            nextTime = df.loc[i+1, 'event time:timestamp']
            currentTime = row['event time:timestamp']
            timeDiff = nextTime - currentTime
            df.at[i, 'time_until_next_event'] = timeDiff.total_seconds()
    
    return df



In [76]:
df_timeDiff = computeTimeDifference(df_event)
df_timeDiff.head(5)

C:\Users\20211321\AppData\Local\Temp\ipykernel_23192\1556820753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_until_next_event'] = 0  # initialize new column with zeros


,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,case:concept:name,concept:name,time:timestamp,@@index,@@case_index,ground_truth_activity,ground_truth_time,time_until_next_event
0,0,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546000+00:00,173688,A_SUBMITTED,2011-01-10 00:38:44.546000+00:00,0,0,A_PARTLYSUBMITTED,2011-01-10 00:38:44.880000+00:00,0.334
1,1,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880000+00:00,173688,A_PARTLYSUBMITTED,2011-01-10 00:38:44.880000+00:00,1,0,A_PREACCEPTED,2011-01-10 00:39:37.906000+00:00,53.026
2,2,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906000+00:00,173688,A_PREACCEPTED,2011-01-10 00:39:37.906000+00:00,2,0,W_Completeren aanvraag,2011-01-10 00:39:38.875000+00:00,0.969
3,3,173688,2011-01-10 00:38:44.546000+00:00,20000,112.0,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875000+00:00,173688,W_Completeren aanvraag,2011-01-10 00:39:38.875000+00:00,3,0,W_Completeren aanvraag,2011-01-10 11:36:46.437000+00:00,39427.562
4,4,173688,2011-01-10 00:38:44.546000+00:00,20000,NaN,W_Completeren aanvraag,START,2011-01-10 11:36:46.437000+00:00,173688,W_Completeren aanvraag,2011-01-10 11:36:46.437000+00:00,4,0,A_ACCEPTED,2011-01-10 11:42:43.308000+00:00,356.871


In [77]:
#new collumn that indexes events within a case

def addEventNumber(df):
    df['event_number'] = 0  # initialize new column with zeros
    
    # iterate over each unique case
    for case in df['case concept:name'].unique():
        case_df = df[df['case concept:name'] == case]  # subset dataframe for current case
        case_df['event_number'] = range(1, len(case_df)+1)  # assign incrementing event numbers
        df.update(case_df)  # update original dataframe with new event numbers
    
    return df



In [78]:
df_eventNum = addEventNumber(df_timeDiff)
df_eventNum.head()


C:\Users\20211321\AppData\Local\Temp\ipykernel_23192\1798810378.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['event_number'] = 0  # initialize new column with zeros
C:\Users\20211321\AppData\Local\Temp\ipykernel_23192\1798810378.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case_df['event_number'] = range(1, len(case_df)+1)  # assign incrementing event numbers
C:\Users\20211321\AppData\Local\Temp\ipykernel_23192\1798810378.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,case:concept:name,concept:name,time:timestamp,@@index,@@case_index,ground_truth_activity,ground_truth_time,time_until_next_event,event_number
0,0.0,173688.0,2011-01-10 00:38:44.546000+00:00,20000.0,112.0,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546000+00:00,173688,A_SUBMITTED,2011-01-10 00:38:44.546000+00:00,0.0,0.0,A_PARTLYSUBMITTED,2011-01-10 00:38:44.880000+00:00,0.334,1.0
1,1.0,173688.0,2011-01-10 00:38:44.546000+00:00,20000.0,112.0,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880000+00:00,173688,A_PARTLYSUBMITTED,2011-01-10 00:38:44.880000+00:00,1.0,0.0,A_PREACCEPTED,2011-01-10 00:39:37.906000+00:00,53.026,2.0
2,2.0,173688.0,2011-01-10 00:38:44.546000+00:00,20000.0,112.0,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906000+00:00,173688,A_PREACCEPTED,2011-01-10 00:39:37.906000+00:00,2.0,0.0,W_Completeren aanvraag,2011-01-10 00:39:38.875000+00:00,0.969,3.0
3,3.0,173688.0,2011-01-10 00:38:44.546000+00:00,20000.0,112.0,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875000+00:00,173688,W_Completeren aanvraag,2011-01-10 00:39:38.875000+00:00,3.0,0.0,W_Completeren aanvraag,2011-01-10 11:36:46.437000+00:00,39427.562,4.0
4,4.0,173688.0,2011-01-10 00:38:44.546000+00:00,20000.0,NaN,W_Completeren aanvraag,START,2011-01-10 11:36:46.437000+00:00,173688,W_Completeren aanvraag,2011-01-10 11:36:46.437000+00:00,4.0,0.0,A_ACCEPTED,2011-01-10 11:42:43.308000+00:00,356.871,5.0


In [79]:
# obsolete method; done alternatively in createNaiveTimePredictor
def getLongestTraceLength(df):
    traceLengths = df.groupby('case concept:name').size()  # count number of events for each trace
    longest_trace_length = traceLengths.max()  # get maximum number of events
    return longest_trace_length

longestTrace = getLongestTraceLength(df_event)


In [80]:
def createNaiveTimePredictor(df):
    # group by event number and calculate mean of time_until_next_event
    df_naiveTimePredictor = df.groupby('event_number')['time_until_next_event'].mean().reset_index()
    return df_naiveTimePredictor


In [83]:
naiveTimePredictor = createNaiveTimePredictor(df_eventNum)
naiveTimePredictor.head(30)

,event_number,time_until_next_event
0,1.0,3.552432e-01
1,2.0,3.921100e+01
2,3.0,1.376018e+05
3,4.0,9.854418e+05
4,5.0,6.094284e+02
5,6.0,5.185245e+03
6,7.0,9.830827e+02
7,8.0,5.648852e+05
8,9.0,3.772099e+05
9,10.0,1.261161e+06


In [84]:
df_eventNum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   eventID                     1000 non-null   float64            
 1   case concept:name           1000 non-null   float64            
 2   case REG_DATE               1000 non-null   datetime64[ns, UTC]
 3   case AMOUNT_REQ             1000 non-null   float64            
 4   event org:resource          809 non-null    float64            
 5   event concept:name          1000 non-null   object             
 6   event lifecycle:transition  1000 non-null   object             
 7   event time:timestamp        1000 non-null   datetime64[ns, UTC]
 8   case:concept:name           1000 non-null   object             
 9   concept:name                1000 non-null   object             
 10  time:timestamp              1000 non-null   datetime64[ns, UT